In [1]:
import numpy as np
import os
from ase import Atoms
from ase.io import read, write
from ase.visualize import view

def pack_existing_chain(gjf_path, n_chains=30, box_size=40.0):
    """
    指定されたGJFファイルの分子を読み込み、
    指定された本数だけ箱の中にランダム（低密度）に配置する関数
    """
    
    # 1. GJFファイルの読み込み
    if not os.path.exists(gjf_path):
        print(f"エラー: ファイルが見つかりません: {gjf_path}")
        return None
        
    try:
        # GJFファイルを読み込む
        # ASEはGaussian形式に対応していますが、フォーマットによっては
        # 'gaussian-in' または単に 'gaussian' と指定が必要な場合があります
        base_chain = read(gjf_path)
        print(f"--- 読み込み成功: {gjf_path}")
        print(f"    分子式: {base_chain.get_chemical_formula()}")
        print(f"    原子数: {len(base_chain)}")
    except Exception as e:
        print(f"読み込みエラー: {e}")
        return None

    # 2. 箱の作成
    system = Atoms(cell=[box_size, box_size, box_size], pbc=True)
    
    print(f"--- 配置開始: {n_chains} 本")
    
    # 3. ランダム配置 (グリッドベースで衝突回避)
    # 衝突を避けるため、広めの箱に配置します
    grid_n = int(np.ceil(n_chains**(1/3))) # 立方根 (例: 27本なら3x3x3)
    spacing = box_size / grid_n
    
    count = 0
    for x in range(grid_n):
        for y in range(grid_n):
            for z in range(grid_n):
                if count >= n_chains: break
                
                chain = base_chain.copy()
                
                # 重心を原点に移動
                chain.center(vacuum=0)
                
                # ランダム回転 (等方的なアモルファスにするため重要)
                chain.rotate(np.random.rand()*360, 'x')
                chain.rotate(np.random.rand()*360, 'y')
                chain.rotate(np.random.rand()*360, 'z')
                
                # グリッド位置へ移動
                target_pos = np.array([x, y, z]) * spacing + spacing/2
                # 少しランダムにずらす（結晶っぽさを消す）
                jitter = (np.random.rand(3) - 0.5) * (spacing * 0.5)
                
                chain.translate(target_pos + jitter)
                
                # 箱からはみ出た部分を反対側へ (PBC)
                # ※注意: 分子が分断されると可視化で見づらいですが、MD的にはOKです
                # ここでは初期配置なので、あえてwrapせず箱を十分大きくする方が安全です
                
                system.extend(chain)
                count += 1
                
    return system

# --- 実行部 ---

# ファイルパス
input_gjf = "/home/jovyan/Kaori/MD/LiB_2/structure/C12H14F12.gjf"
output_cif = "pvdf_amorphous_initial.cif"

# 設定: 鎖の数と箱の大きさ
# 30本配置すると、総原子数は 38 * 30 = 1140原子程度になります
# 箱サイズ40Aだと初期密度は低く、安全に圧縮を開始できます
initial_atoms = pack_existing_chain(gjf_path=input_gjf, n_chains=30, box_size=40.0)

if initial_atoms:
    # 密度の確認
    vol_cm3 = initial_atoms.get_volume() * (1e-8)**3
    mass_g = initial_atoms.get_masses().sum() / 6.022e23
    density = mass_g / vol_cm3
    
    print(f"--- 作成完了 ---")
    print(f"総原子数: {len(initial_atoms)}")
    print(f"初期密度: {density:.3f} g/cm3 (目標 ~1.78)")
    
    # 保存
    initial_atoms.write(output_cif)
    print(f"保存しました: {output_cif}")
    print("-> 次のステップ: この構造をNPT計算(高温・高圧)にかけて圧縮してください。")

--- 読み込み成功: /home/jovyan/Kaori/MD/LiB_2/structure/C12H14F12.gjf
    分子式: C12H14F12
    原子数: 38
--- 配置開始: 30 本
--- 作成完了 ---
総原子数: 1140
初期密度: 0.301 g/cm3 (目標 ~1.78)
保存しました: pvdf_amorphous_initial.cif
-> 次のステップ: この構造をNPT計算(高温・高圧)にかけて圧縮してください。


In [1]:
import os
import numpy as np
from pathlib import Path
from ase.io import read, write
from ase import units
import numpy as np
from ase.io import read, write, Trajectory
from ase.constraints import FixAtoms
from ase import units
# from ase.md.langevin import Langevin # <-- 削除
# from ase.md.velocitydistribution import MaxwellBoltzmannDistribution # <-- 削除
from pathlib import Path
import logging # MDヘルパー関数用にインポート
from time import perf_counter # MDヘルパー関数用にインポート
from typing import Dict, Any # MDヘルパー関数用にインポート
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import numpy as np
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import (ASEMDSystem, MDFeature, MDExtensionBase,
                                            NVTBerendsenIntegrator, NPTIntegrator, NPTBerendsenIntegrator)

# from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import numpy as np
import pandas as pd
from pathlib import Path
from ase import units
from ase.build import add_vacuum
import re
from matlantis_features.features.common.opt import FireLBFGSASEOptFeature

def run_matlantis_optimization(
    atoms,
    trajectory_path: str,
    fmax: float = 0.05,
    name: str = "structure",
    model_version: str = 'v7.0.0',
    calc_mode: str = 'CRYSTAL_U0',
    n_run: int = 5000,
    show_progress_bar: bool = True,
):
    """
    Matlantisを使って構造最適化を実行する
    (内容は変更ありません)
    """
    print(f"   -> Matlantis最適化開始 ({name}, fmax = {fmax}) ...")

    matlantis_atoms = MatlantisAtoms(atoms)
    estimator_function = pfp_estimator_fn(
        model_version=model_version,
        calc_mode=calc_mode
    )
    position_optimizer = FireLBFGSASEOptFeature(
        estimator_fn=estimator_function,
        filter=False,
        trajectory=str(trajectory_path),
        n_run=n_run,
        fmax=fmax,
        show_progress_bar=show_progress_bar
    )

    try:
        result = position_optimizer(matlantis_atoms)
        optimized_atoms = result.atoms.ase_atoms
        final_energy = result.output.energy_log[-1]
        print(f"   -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")

        # XYZ形式でも保存
        xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
        print(f"   -> 💾 最適化後の構造を保存します: {xyz_save_path.name}")
        write(str(xyz_save_path), optimized_atoms)

        return optimized_atoms

    except Exception as e:
        print(f"   -> ❌ 最適化中にエラーが発生: {e}")
        return None


"""
MD (分子動力学) シミュレーション関連のユーティリティ関数

このモジュールには以下の機能が含まれます：
- Integrator選択とMD実行
- MDログの記録
"""

import logging
from time import perf_counter
from typing import Dict, Any

import numpy as np
from ase import units

from matlantis_features.features.md import (
    MDFeature,
    ASEMDSystem,
    MDExtensionBase,
    VelocityVerletIntegrator,
    LangevinIntegrator,
    NVTBerendsenIntegrator,
    NPTBerendsenIntegrator,
    NPTIntegrator,
)
from matlantis_features.utils.calculators import pfp_estimator_fn


class PrintWriteLog(MDExtensionBase):
    """
    MDシミュレーションのログを記録・出力するクラス

    Args:
        fname (str): ログファイル名（拡張子なし）
        dirout (str): 出力ディレクトリ（デフォルト: '.'）
        stdout (bool): 標準出力に表示するか（デフォルト: False）
    """

    def __init__(self, fname: str, dirout: str = '.', stdout: bool = False):
        self.fname = fname
        self.dirout = dirout
        self.t_start = perf_counter()
        self.stdout = stdout

    def __call__(self, system, integrator):
        n_step = system.current_total_step
        sim_time = system.current_total_time / 1000  # ps
        E_tot = system.ase_atoms.get_total_energy()
        E_pot = system.ase_atoms.get_potential_energy()
        E_kin = system.ase_atoms.get_kinetic_energy()
        temp = system.ase_atoms.get_temperature()
        density = system.ase_atoms.get_masses().sum() / units.mol / (
            system.ase_atoms.cell.volume * (1e-8**3)
        )
        calc_time = (perf_counter() - self.t_start) / 60.  # min.

        # ヘッダー書き込み
        if n_step == 0:
            hdr = 'step,time[ps],E_tot[eV],E_pot[eV],E_kin[eV],'
            hdr += 'T[K],density[g/cm3],calc_time[min]'
            with open(f'{self.dirout}/{self.fname}.log', 'w') as f_log:
                f_log.write(f'{hdr}\n')

        # 結果の書き込みと出力
        line = f'{n_step:8d},{sim_time:7.2f},'
        line += f'{E_tot:11.4f},{E_pot:11.4f},{E_kin:9.4f},'
        line += f'{temp:8.2f},{density:7.3f},{calc_time:8.2f}'
        with open(f'{self.dirout}/{self.fname}.log', 'a') as f_log:
            f_log.write(f'{line}\n')
        if self.stdout:
            print(line)


def select_integrator(
    integrator_type: str,
    timestep: float,
    temperature: float,
    pressure: float = 101325.0,
):
    """
    指定された文字列に応じて、対応するIntegratorオブジェクトを返す

    Args:
        integrator_type (str): Integratorの種類
            - "NVE": Velocity Verlet (エネルギー保存)
            - "Langevin": Langevin dynamics (NVT)
            - "NVT_Berendsen": Berendsen thermostat (NVT)
            - "NPT_Berendsen": Berendsen thermostat and barostat (NPT)
            - "NPT": Nosé-Hoover thermostat and Parrinello-Rahman barostat (NPT)
        timestep (float): タイムステップ (fs)
        temperature (float): 温度 (K)
        pressure (float): 圧力 (Pa)、NPT系でのみ使用

    Returns:
        Integratorオブジェクト
    """
    print(f"--- Integrator: '{integrator_type}' を選択しました ---")

    # 圧力をASE unitsに変換
    pressure_au = pressure * units.Pascal

    if integrator_type == "NVE":
        return VelocityVerletIntegrator(timestep=timestep)

    elif integrator_type == "Langevin":
        return LangevinIntegrator(
            timestep=timestep,
            temperature=temperature,
            friction=0.002
        )

    elif integrator_type == "NVT_Berendsen":
        return NVTBerendsenIntegrator(
            timestep=timestep,
            temperature=temperature,
            taut=100.0 * units.fs
        )

    elif integrator_type == "NPT_Berendsen":
        return NPTBerendsenIntegrator(
            timestep=timestep,
            temperature=temperature,
            pressure=pressure_au,
            taut=100.0 * units.fs,
            taup=1000.0 * units.fs
        )

    elif integrator_type == "NPT":
        return NPTIntegrator(
            timestep=timestep,
            temperature=temperature,
            pressure=pressure_au,
            ttime=25.0 * units.fs,
            pfactor=75.0 * units.fs
        )

    else:
        raise ValueError(f"エラー: 指定されたIntegrator '{integrator_type}' は不正です。")


def run_md_simulation(
    atoms,
    integrator_type: str,
    temperature: float,
    n_steps: int,
    timestep: float = 1.0,
    pressure: float = 101325.0,
    traj_file: str = "md.traj",
    traj_freq: int = 50,
    logger_interval: int = 50,
    model_version: str = 'v7.0.0',
    calc_mode: str = 'CRYSTAL_U0',
    show_progress: bool = True,
    show_logger: bool = True,
):
    """
    MDシミュレーションを実行する汎用関数

    Args:
        atoms: ASE Atomsオブジェクト
        integrator_type (str): Integratorの種類
        temperature (float): 温度 (K)
        n_steps (int): 実行ステップ数
        timestep (float): タイムステップ (fs)
        pressure (float): 圧力 (Pa)
        traj_file (str): トラジェクトリファイル名
        traj_freq (int): トラジェクトリ保存間隔
        logger_interval (int): ログ出力間隔
        model_version (str): PFPモデルバージョン
        calc_mode (str): PFP計算モード
        show_progress (bool): 進捗バー表示
        show_logger (bool): ログ表示

    Returns:
        MDシミュレーション結果
    """
    # ロガー設定
    logger = logging.getLogger("matlantis_features")
    logger.setLevel(logging.INFO)

    # PFP計算器の準備
    estimator_fn = pfp_estimator_fn(model_version=model_version, calc_mode=calc_mode)

    # MDシステムの初期化
    system = ASEMDSystem(atoms)
    system.init_temperature(
        temperature=temperature,
        stationary=True,
        rng=np.random.RandomState(seed=12345)
    )
    print(f"システムを {temperature} K に初期化しました。")

    # Integratorの選択と準備
    integrator = select_integrator(integrator_type, timestep, temperature, pressure)

    # MDFeatureの設定
    md = MDFeature(
        integrator=integrator,
        n_run=n_steps,
        show_progress_bar=show_progress,
        show_logger=show_logger,
        logger_interval=logger_interval,
        traj_file_name=traj_file,
        traj_freq=traj_freq,
        estimator_fn=estimator_fn,
    )

    # MDシミュレーションの実行
    print(f"\n--- MDシミュレーションを開始します ({n_steps} ステップ) ---")
    md_results = md(system)

    print("\n--- MDシミュレーションが正常に完了しました ---")
    print(f"最終的なトラジェクトリは '{traj_file}' に保存されました。")
    print(f"最終ステップ: {system.current_total_step}, 最終時間: {system.current_time:.2f} fs")

    return md_results


def run_constant_temp_md(md_params: Dict[str, Any]):
    """
    一定温度でNVT-MDを実行するための関数

    Args:
        md_params (dict): MDパラメータを含む辞書
            - atoms: ASE Atomsオブジェクト
            - temperature: 温度 (K)
            - timestep: タイムステップ (fs)
            - n_run: 実行ステップ数
            - model_version: PFPモデルバージョン
            - calc_mode: PFP計算モード
            - dirout: 出力ディレクトリ
            - fname: ファイル名
            - logger_interval: ログ間隔
            - traj_freq: トラジェクトリ保存間隔

    Returns:
        MDシミュレーション結果
    """
    estimator_fn = pfp_estimator_fn(
        model_version=md_params['model_version'],
        calc_mode=md_params['calc_mode'],
    )
    system = ASEMDSystem(
        atoms=md_params['atoms'],
        step=0,
        time=0.0,
    )
    integrator = NVTBerendsenIntegrator(
        timestep=md_params['timestep'],
        temperature=md_params['temperature'],
        taut=100.,
        fixcm=True,
    )
    md = MDFeature(
        integrator=integrator,
        n_run=md_params['n_run'],
        show_progress_bar=True,
        show_logger=False,
        logger_interval=md_params['logger_interval'],
        estimator_fn=estimator_fn,
        traj_file_name=f"{md_params['dirout']}/{md_params['fname']}.traj",
        traj_freq=md_params['traj_freq'],
    )

    print(f"\n--- MD計算を開始: {md_params['fname']} ---")
    print(f"ステップ数: {md_params['n_run']} ({md_params['n_run'] * md_params['timestep']/1000:.1f} ps), 温度: {md_params['temperature']:.2f} K")

    md_results = md(
        system=system,
        extensions=[
            (PrintWriteLog(fname=md_params['fname'], dirout=md_params['dirout'], stdout=True),
             md_params['logger_interval'])
        ]
    )
    print(f"--- MD計算が完了: {md_params['fname']} ---")
    return md_results


In [1]:
pip install matlantis_features

Looking in indexes: https://pypi.org/simple, http://pypi.artifact.svc:8080/simple

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
from pathlib import Path
from ase.io import read, write
from ase import units

# --- Matlantis Features 関連のインポート ---
# try:
#     # Core features
#     from matlantis_features.features.common import MatlantisAtoms
#     from matlantis_features.utils.calculators import pfp_estimator_fn
    
#     # Optimization feature (ご指定のクラス)
#     from matlantis_features.features.opt import FireLBFGSASEOptFeature
    
#     # MD feature components (後のMD用)
#     from matlantis_features.features.md import ASEMDSystem
# except ImportError:
#     print("エラー: matlantis_features が見つかりません。")
#     exit()

# --- MD実行関数のインポート ---
# (ご自身の lib2_utils がある前提)


def get_current_density(atoms):
    """現在の密度(g/cm3)を計算"""
    vol_cm3 = atoms.get_volume() * (1e-8)**3
    mass_g = atoms.get_masses().sum() / 6.022e23
    return mass_g / vol_cm3

if __name__ == "__main__":
    
    # === 設定 ===
    input_file = "pvdf_amorphous_initial.cif"
    output_file = "pvdf_amorphous_equilibrated.cif"
    
    model_version = 'v7.0.0'
    calc_mode = 'CRYSTAL_U0'
    
    if not os.path.exists(input_file):
        print(f"エラー: 入力ファイル {input_file} が見つかりません。")
        exit()

    # 1. 構造読み込み
    print(f"--- 1. 読み込み: {input_file} ---")
    atoms = read(input_file)
    print(f"  原子数: {len(atoms)}")
    print(f"  初期密度: {get_current_density(atoms):.3f} g/cm3")

    # 2. 構造最適化 (FireLBFGSASEOptFeature を使用)
    print("\n--- 2. 構造最適化 (FireLBFGS) ---")
    
    # --- ★ ご指定のコードブロックに基づいた実装 ★ ---
    trajectory_path = Path("opt_init.traj")
    
    # MatlantisAtomsへの変換
    matlantis_atoms = MatlantisAtoms(atoms)
    
    # Estimatorの作成
    estimator_function = pfp_estimator_fn(
        model_version=model_version,
        calc_mode=calc_mode
    )
    
    # Optimizer Featureの設定
    position_optimizer = FireLBFGSASEOptFeature(
        estimator_fn=estimator_function,
        filter=False, # 座標のみ最適化 (セルは固定)
        trajectory=str(trajectory_path),
        n_run=500,    # 最大ステップ数 (必要に応じて増減)
        fmax=0.1,     # 収束判定力
        show_progress_bar=True
    )

    # 実行
    try:
        # result = position_optimizer(matlantis_atoms)
        # optimized_atoms = result.atoms.ase_atoms # ASE Atomsを取り出す
        optimized_atoms=read("opt_init.xyz")
        # エネルギーログの取得 (リストが空でないか確認)
        if result.output.energy_log:
            final_energy = result.output.energy_log[-1]
            print(f"   -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
        else:
            print(f"   -> ✔️ 最適化完了 (エネルギーログなし)")

        # XYZ形式でも保存 (ご指定通り)
        xyz_save_path = trajectory_path.with_suffix('.xyz')
        print(f"   -> 💾 最適化後の構造を保存します: {xyz_save_path.name}")
        write(str(xyz_save_path), optimized_atoms)
        
        # 後のMDのためにatomsを更新
        atoms = optimized_atoms.copy()

    except Exception as e:
        print(f"最適化中にエラーが発生しました: {e}")
        exit()
    # ---------------------------------------------------


    # 3. 高温NPT圧縮 (600K, 1 atm)
    print("\n--- 3. 高温NPT緩和 (600K) - 圧縮プロセス ---")
    
    # 密度が低い状態から圧縮するため、最初はタイムステップを小さく
    run_md_simulation(
        atoms=atoms,
        integrator_type="NPT_Berendsen",
        temperature=600.0,
        pressure=101325.0, # 1 atm
        n_steps=40000,     # 20 ps
        timestep=0.5,      # 安全のため0.5fs
        traj_file="md_npt_600K.traj",
        traj_freq=200,
        logger_interval=200,
        model_version=model_version,
        calc_mode=calc_mode,
        show_progress=True
    )
    
    dens_600k = get_current_density(atoms)
    print(f"  600K緩和後の密度: {dens_600k:.3f} g/cm3")

    # 4. 冷却・平衡化NPT (300K, 1 atm)
    print("\n--- 4. 冷却・平衡化NPT (300K) ---")
    
    run_md_simulation(
        atoms=atoms,
        integrator_type="NPT_Berendsen",
        temperature=300.0,
        pressure=101325.0, # 1 atm
        n_steps=40000,     # 20 ps
        timestep=0.5, 
        traj_file="md_npt_300K.traj",
        traj_freq=200,
        logger_interval=200,
        model_version=model_version,
        calc_mode=calc_mode,
        show_progress=True
    )

    # 5. 最終確認と保存
    final_density = get_current_density(atoms)
    print(f"\n--- 5. 完了 ---")
    print(f"  最終密度: {final_density:.3f} g/cm3")
    
    if 1.70 < final_density < 1.90:
        print("  >> 成功: 妥当な密度が得られました。")
    else:
        print("  >> 注意: 密度が目標範囲(1.78付近)から外れている可能性があります。")

    write(output_file, atoms)
    print(f"  構造を保存しました: {output_file}")

In [ ]:
# === 高圧NPTシミュレーション (1000気圧, 473K) ===
# z方向のみのセル変形を許可するNPT MD

print("\n--- 高圧NPTシミュレーション (1000 atm, 473K) ---")

# 入力ファイルの読み込み（前のステップの結果を使用）
# slab_opt_path = "pvdf_amorphous_equilibrated.cif"  # 前のステップの出力
# ase_atoms = read(slab_opt_path)

# ここでは前のステップのatomsをそのまま使用
ase_atoms = atoms.copy()

# 制約の設定（z座標が3Å以下の原子を固定）
# indices = [atom.index for atom in ase_atoms if atom.position[2] <= 3]
# if indices:
#     c = FixAtoms(indices=indices)
#     ase_atoms.set_constraint(c)
#     print(f"  {len(indices)} 個の原子を固定しました")

# MatlantisAtomsへの変換
atoms_mat = MatlantisAtoms.from_ase_atoms(ase_atoms)
# atoms_mat.rotate_atoms_to_upper()  # 必要に応じて

# === NPTパラメータ設定 ===
t_step = 1.0        # fs
temperature = 473.0  # K

# 圧力設定（正しいunitsの使用）
# オプション1: Pascalを使用（最も明確）
pressure = 1000 * 101325 * units.Pascal  # 1000 atm = 101.325 MPa

# オプション2: GPaを使用する場合
# pressure = 0.101325 * units.GPa  # 1000 atm

# オプション3: barを使用する場合
# pressure = 1000 * 1.01325 * units.bar  # 1000 atm

print(f"  圧力設定: 1000 atm = {pressure / units.GPa:.6f} GPa")
print(f"  温度: {temperature} K")

# MDシステムの初期化
system = ASEMDSystem(atoms_mat.ase_atoms)
system.init_temperature(temperature)

# NPT Integratorの設定
integrator = NPTIntegrator(
    timestep=t_step,
    temperature=temperature,
    pressure=pressure,  # 既にunits変換済み
    ttime=20.0 * units.fs,
    pfactor=2e6 * units.GPa * (units.fs**2),
    mask=np.array([[0, 0, 0], [0, 0, 0], [0, 0, 1]])  # z方向のみセル変形を許可
)

# エスティメータの準備
estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='CRYSTAL_U0')

# トラジェクトリファイル名
npt_1000atm_path = "md_npt_1000atm_473K.traj"

# MD Feature の設定
md = MDFeature(
    integrator, 
    n_run=75000,  # 75 ps
    traj_file_name=npt_1000atm_path,
    traj_freq=100, 
    estimator_fn=estimator_fn,
    show_progress_bar=True
)

# 拡張機能（セル形状の出力）
class PrintCellShape(MDExtensionBase):
    """セルの形状を出力する拡張機能"""
    def __call__(self, system, integrator):
        cell = system.ase_atoms.get_cell()
        volume = system.ase_atoms.get_volume()
        density = get_current_density(system.ase_atoms)
        step = system.current_total_step
        time_ps = system.current_total_time / 1000.0
        print(f"  Step {step:6d} ({time_ps:6.2f} ps): "
              f"V={volume:8.2f} Å³, ρ={density:.3f} g/cm³, "
              f"c={cell[2,2]:.3f} Å")

info = PrintCellShape()

# MD実行
print(f"  MDを開始します: {75000} ステップ (75 ps)")
md(system, extensions=[(info, 1000)])

# 最終状態の保存
atoms_final = system.ase_atoms
final_density = get_current_density(atoms_final)
print(f"\n  最終密度: {final_density:.3f} g/cm³")

# 結果の保存
output_1000atm = "pvdf_npt_1000atm_473K_final.cif"
write(output_1000atm, atoms_final)
print(f"  最終構造を保存: {output_1000atm}")
print(f"  トラジェクトリ: {npt_1000atm_path}")
